In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [96]:
# Read all data into a list of dataframes

data = pd.read_csv('C:/Users/alanc/Historical Stock Prices/Data.csv')
data['S&P 500 Growth']

0     -0.724860
1      0.877148
2      2.073923
3      2.954354
4      1.694879
         ...   
257   -2.934253
258    1.327823
259    3.840532
260   -2.575776
261   -0.216284
Name: S&P 500 Growth, Length: 262, dtype: float64

In [97]:
# Do one-hot coding for INDUSTRY feature
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(data['S&P 500 Growth'])
classes = lb.classes_
for df in data:
    onehot = lb.transform(df['S&P 500 Growth'])
    for i in range(len(classes)):
        df[classes[i]] = onehot[:,i]

ValueError: Unknown label type: (0     -0.724860
1      0.877148
2      2.073923
3      2.954354
4      1.694879
         ...   
257   -2.934253
258    1.327823
259    3.840532
260   -2.575776
261   -0.216284
Name: S&P 500 Growth, Length: 262, dtype: float64,)

In [39]:
from sklearn.linear_model import LinearRegression

# This is featurelsit for samples
featurelist = ['Inflation','Unemployment','GDP Growth','Interest Rate','S&P 500 Growth']

for df in dfs:

    # Fill Nan where ESTU==0 ,which means thes samples don't have alpha factor
    df_filled = df.fillna(0)
    
    # OLS regression
    lr = LinearRegression(fit_intercept=False)
    lr.fit(df_filled[featurelist].values,df_filled['R'])
    Prd = lr.predict(df_filled[featurelist].values)
    Res = df_filled['R'].values - Prd
    
    # Store Residuals into dataframe
    df['Residuals'] = Res

NameError: name 'dfs' is not defined

In [7]:
# Set of all stocks in all files
Allindex = set()
for df in dfs:
    Allindex = Allindex | set(df.index)

NameError: name 'dfs' is not defined

In [ ]:
# Create a dataframe storing residuals of each day and each stock
# Each row is a stock residual time series. There exists NaN.
Resnamelist = []
for i in range(len(dfs)):
    Resnamelist.append('Res%d'%i)

Res_df = pd.DataFrame(np.nan,index = Allindex, columns = Resnamelist)

for i in range(len(dfs)):
    name = 'Res%d'%i
    Res_df.loc[list(dfs[i].index),name] = dfs[i]['Residuals']

In [8]:
# EWMA volitality
Sigma = Res_df.ewm(halflife=50,axis=1).var()
Sigma = Sigma.fillna(method='backfill',axis=1)

NameError: name 'Res_df' is not defined

In [9]:
# OLS for ESTU=1
paralist_ols = []

for df in dfs:
    df = df[df['ESTU']==1]
    lr = LinearRegression(fit_intercept=False)
    lr.fit(df[featurelist].values,df['R'])
    para = lr.coef_
    paralist_ols.append(para) 

# Calculate mean factor return for all days
paralist_ols = np.array(paralist_ols)
f_OLS = np.mean(paralist_ols,axis =0)

NameError: name 'dfs' is not defined

In [10]:
# WLS for ESTU=1
paralist_wls = []

for i in range(len(dfs)):
    df = dfs[i][dfs[i]['ESTU']==1]
    weight = 1/Sigma.loc[df.index,'Res%d'%i]
    Y = df['R'].values*weight
    weight = np.tile(weight,(len(featurelist),1)).T
    X = df[featurelist].values*weight
    lr = LinearRegression(fit_intercept=False)
    lr.fit(X,Y)
    para = lr.coef_
    paralist_wls.append(para)

# Calculate mean factor return for all days
paralist_wls = np.array(paralist_wls)
f_WLS = np.mean(paralist_wls,axis =0)

NameError: name 'dfs' is not defined

In [11]:
# featurelist = ['BETA','MOMENTUM','SIZE','VOL','TD','TIR']+list(classes)
# Print out average factors 'TD' and 'TIR'
toPrint = pd.DataFrame([f_OLS[4:6],f_WLS[4:6]],index = ['OLS','WLS'],columns=['TD','TIR'])
print(toPrint)

NameError: name 'f_OLS' is not defined